In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import string
import re
import gensim
from gensim.models import Word2Vec
from gensim.models import FastText
import gensim.downloader as api
print((gensim.__version__))  # needed 3.8.3   -> pip install gensim==3.8.3
import pickle
import time

4.1.2


In [2]:
def tokenized_column(string_col
                    ):  
    tokenized_list=[]
    for i in list(string_col):
        li = list(i.split(' '))
        tokenized_list.append(li)
    return tokenized_list

# EMBEDDINGS

In [3]:
df_all = pd.read_csv("outputs/df_sw_tok_low_punc_lemm_v7.csv")

In [4]:
len(df_all)

476175

In [5]:
df_all.isnull().sum()

Unnamed: 0              0
Year                  711
Month                 711
abstract                0
doi                     0
cord_uid                0
journal             40225
license                 0
authors              1986
len                     0
language                0
abstract_cleaned        0
dtype: int64

## A) Train W2V with articles published between years

In [6]:
to_years = [2017,2018,2019,2020,2021]
diff_list = [1]

for diff in diff_list:
    for to_year in to_years:
        print("Training started................")
        print("Max publ. year: "+str(to_year))
        print("Min publ. year: "+str(to_year-diff))
        df_all_to_year = df_all[ (df_all['Year']<=to_year) & (df_all['Year']>=to_year-diff) ]
        print("Count of articles: " + str(len(df_all_to_year)))
        model = gensim.models.word2vec.Word2Vec(tokenized_column(df_all_to_year["abstract_cleaned"].astype(str)),
                                        vector_size=100, 
                                        window = 10,
                                        min_count=2,
                                        sg=1,
                                        seed=1
                                               ) 
        model.save("outputs/w2v/w2v_published_between_"+str(to_year-diff) + " and "+ str(to_year)+".model")

Training started................
Max publ. year: 2017
Min publ. year: 2016
Count of articles: 12589
Training started................
Max publ. year: 2018
Min publ. year: 2017
Count of articles: 13036
Training started................
Max publ. year: 2019
Min publ. year: 2018
Count of articles: 14503
Training started................
Max publ. year: 2020
Min publ. year: 2019
Count of articles: 132422
Training started................
Max publ. year: 2021
Min publ. year: 2020
Count of articles: 350176
